In [1]:
#importing neccessary libraries
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import csv

In [2]:
#getting the links for all the pages with the school names
main_link= "https://www.fcps.edu/schools-centers?keywords=&items_per_page=50"
new_page_link=main_link.split("?")
linki= new_page_link[0]
linki


'https://www.fcps.edu/schools-centers'

In [3]:
full_link= []
page= requests.get("https://www.fcps.edu/schools-centers?keywords=&items_per_page=50")
xy= bs(page.content, "lxml")
next_page= xy.find_all("a", "pager__link")
for page in next_page:
    half_link= (page['href'])
    full_link.append(linki + half_link)
    
full_link

['https://www.fcps.edu/schools-centers?items_per_page=50&keywords=&page=1',
 'https://www.fcps.edu/schools-centers?items_per_page=50&keywords=&page=2',
 'https://www.fcps.edu/schools-centers?items_per_page=50&keywords=&page=3',
 'https://www.fcps.edu/schools-centers?items_per_page=50&keywords=&page=4',
 'https://www.fcps.edu/schools-centers?items_per_page=50&keywords=&page=5',
 'https://www.fcps.edu/schools-centers?items_per_page=50&keywords=&page=1',
 'https://www.fcps.edu/schools-centers?items_per_page=50&keywords=&page=5']

In [4]:
#getting the website of the each school available on these pages
main_website= "https://www.fcps.edu/"
scho_websites=[]

def sch_websites(link):
    page= requests.get(link)
    soup= bs(page.content, "lxml")
    web_div= soup.find_all("div", class_= "node__title")
    for x in web_div:
        scho_websites.append(main_website + x.a['href'])

for linkk in full_link:
    sch_websites(linkk)

In [7]:
#scho_websites

In [8]:
#getting the link of the staff directory of each school to get access to their teachers and employess

staff_dict_link= []
def staff(school):
    page= requests.get(school)
    soup= bs(page.content, "lxml")
    try:
        staff_dir= soup.find_all(href=re.compile("staff-directory"))
        staff_dir= staff_dir[0]
        staff_dict_link.append(staff_dir['href']) 
    except Exception as e:
        pass

    
for link in scho_websites:
    staff(link)

In [7]:
#staff_dict_link

['https://www.fcps.edu/DogwoodES//staff-directory',
 'https://dranesvillees.fcps.edu/staff-directory',
 'https://eagleviewes.fcps.edu/staff-directory',
 'https://edisonacademy.fcps.edu//staff-directory',
 'https://edisonhs.fcps.edu/staff-directory',
 'https://fairfaxacademy.fcps.edu/staff-directory',
 'https://fairfaxhs.fcps.edu/staff-directory',
 'https://fairfaxvillaes.fcps.edu//staff-directory',
 'https://fairhilles.fcps.edu/staff-directory',
 'https://fairviewes.fcps.edu//staff-directory',
 'https://fallschurchacademy.fcps.edu//staff-directory',
 'https://fallschurchhs.fcps.edu/staff-directory',
 'https://flinthilles.fcps.edu//staff-directory',
 'https://florises.fcps.edu/staff-directory',
 'https://forestedgees.fcps.edu/staff-directory',
 'https://forestdalees.fcps.edu/staff-directory',
 'https://forestvillees.fcps.edu/staff-directory',
 'https://fortbelvoires.fcps.edu//staff-directory',
 'https://fortbelvoires.fcps.edu//staff-directory',
 'https://forthuntes.fcps.edu/staff-direct

In [17]:
#getting the teachers names and position details

per_page_50= "?keywords=&field_last_name_from=&field_last_name_to=&items_per_page=50"

teacher_details= {} 
def teachers_data(fulllink):
    page= requests.get(fulllink)
    soup= bs(page.content, "lxml")

    table= soup.find("table", class_= "cols-3")
    table_row= table.find_all("tr")[1:]
    for row in table_row:
        username= ""
        for data in row.find_all("td"):
            a_tags= data.find_all("a")
            if a_tags:
                for a_tag in a_tags:
                    username += a_tag.text
            else:
                position= data.text
                teacher_details.update({username:position})
    
for link in staff_dict_link:
    teachers_data(link+ per_page_50)


In [60]:
#converting to dataframe to have a good look
df = pd.DataFrame(list(teacher_details.items()),columns = ['Name','Position'])

In [61]:
print(df)

                  Name                               Position
0     AbdelgaderNaglaa               Public Health Attendant 
1         AdamsColleen                Grades 1-3 Teacher, ES 
2         AlbertRachel          Title I Resource Teacher, ES 
3           AliJanelle              Kindergarten Teacher, ES 
4           AllenMison                Grades 1-3 Teacher, ES 
...                ...                                    ...
7495         ClarkAlex  History & Social Studies Teacher, HS 
7496       ClinchJulie          Educational Interpreter I/II 
7497      CoganWilliam       Marketing Education Teacher, HS 
7498         ColeJayme         Multiple Disabilities Teacher 
7499   Cole-KellyKatie                   Biology Teacher, HS 

[7500 rows x 2 columns]


In [78]:
#processing data
dataframe = pd.DataFrame({
    'Name':df.Name, 
    'Position': df.Position})

In [79]:
dataframe

,Name,Position
0,AbdelgaderNaglaa,Public Health Attendant
1,AdamsColleen,"Grades 1-3 Teacher, ES"
2,AlbertRachel,"Title I Resource Teacher, ES"
3,AliJanelle,"Kindergarten Teacher, ES"
4,AllenMison,"Grades 1-3 Teacher, ES"
...,...,...
7495,ClarkAlex,"History & Social Studies Teacher, HS"
7496,ClinchJulie,Educational Interpreter I/II
7497,CoganWilliam,"Marketing Education Teacher, HS"
7498,ColeJayme,Multiple Disabilities Teacher


In [80]:
#exporting to a csv file
dataframe.to_csv('School_details.csv', columns=['Name', 'Position'], index=False)